In [ ]:
class UnionFind:

    def __init__(self, n):
        self.parent = []
        self.rank = rank = [1] * (n + 1)
        for i in range(n + 1):
            self.parent.append(i)

    def find_parent(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find_parent(self.parent[x])
        return self.parent[x]

    def union(self, v1, v2):
        p1, p2 = self.find_parent(v1), self.find_parent(v2)
        if p1 == p2:
            return False
        elif self.rank[p1] > self.rank[p2]:
            self.parent[p2] = p1
            self.rank[p1] = self.rank[p1] + self.rank[p2]
        else:
            self.parent[p1] = p2
            self.rank[p2] = self.rank[p2] + self.rank[p1]

        return True

In [ ]:
def last_day_to_cross(row, col, cells):

    # Variables to track the top and bottom rows
    connections = UnionFind(row * col + 2)
    # Left, Right, Top, Bottom neighbours
    neighbours = [[0, -1], [0, 1], [-1, 0], [1, 0]]
    # Subtract -1 from all the values in cells array i.e.
    # 2 -> 1 and 1-> 0
    # to make it 0-based
    cells = [(r - 1, c - 1) for r, c in cells]
    # Initialize a grid with the same dimensions as the matrix
    # and mark all the cells array as being filled with water
    grid = [[1] * col for _ in range(row)]

    # Start backtracking from last cell in cells array
    for i in range(len(cells) - 1, -1, -1):
        r, c = cells[i][0], cells[i][1]
        print("\n\tFor cell: [", r, "][", c, "]", sep="")
        # Make the current cell as land in the given cells array
        grid[r][c] = 0
        for r_neigh, c_neigh in neighbours:
            first_ind = connections.find_index(r + r_neigh, c + c_neigh, col)
            # Verify that traversing is within the limits of the grid
            if r + r_neigh >= 0 and r + r_neigh < row and \
                c + c_neigh >= 0 and c + c_neigh < col and \
                    grid[r + r_neigh][c + c_neigh] == 0:
                # check whether the neighbor is land or water
                # if we can move in any direction then we'll
                # find union of both indices
                print("\tNeighbor: [", r + r_neigh, "][",
                      c + c_neigh, "]", sep="")
                second_ind = connections.find_index(r, c, col)
                print("\tConnecting", first_ind, second_ind, "indices")
                connections.union(first_ind, second_ind)
        if r == 0:
            print("\tConnecting the top row with the "
                  "first index in 1D arry...")
            connections.union(0, connections.find_index(r, c, col))
            print("\n")
        if r == row - 1:
            print("\tConnecting the bottom row with the "
                  "last index in 1D arry...")
            rr = row * col + 1
            cc = connections.find_index(r, c, col)
            print("\tConnecting", rr, cc, "indices")
            connections.union(rr, cc)  # last index and current index
            print("\n")

        f1 = connections.find(0)
        f2 = connections.find(row*col + 1)
        print("\n\tUpdated array after finding the parent of"
              " first and last indices:", connections.parent)
        if f1 == f2:
            return i

In [ ]:
from collections import defaultdict


def min_malware_spread(graph, initial):
    # Stores the length of the graph
    length = len(graph)
    # Calls UnionFind constructor
    union_find = UnionFind(length)
    # Find all the connected components of the graph
    for x in range(length):
        for y in range(length):
            if graph[x][y]:
                union_find.union(x, y)

    infected = defaultdict(int)

    # Count the number of initial infected nodes each connected component has
    for x in initial:
        infected[union_find.find(x)] += 1

    maximum_size, candidate_node = 0, min(initial)
    # Count all the infected nodes each connected component has
    for i in initial:
        infection_count = infected[union_find.find(i)]
        component_size = union_find.sizes[union_find.find(i)]

        if infection_count != 1:
            continue
        # Return the candidate node from largest length connected component
        if component_size > maximum_size:
            maximum_size = component_size
            candidate_node = i
        elif component_size == maximum_size and i < candidate_node:
            candidate_node = i
    return candidate_node

In [ ]:
def num_islands(grid):
    if not grid:
        return 0

    cols = len(grid[0])
    rows = len(grid)
    union_find = UnionFind(grid)

    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == '1':
                grid[r][c] = '0'

                if r - 1 >= 0 and grid[r-1][c] == "1":
                    union_find.union(r * cols + c, (r-1) * nc + c)
                if r + 1 < rows and grid[r+1][c] == "1":
                    union_find.union(r * cols + c, (r+1) * cols + c)
                if c - 1 >= 0 and grid[r][c-1] == "1":
                    union_find.union(r * cols + c, r * cols + c - 1)
                if c + 1 < cols and grid[r][c+1] == "1":
                    union_find.union(r * cols + c, r * cols + c + 1)

    count = union_find.get_count()
    return count

In [ ]:
def redundant_connection(edges):
    graph = UnionFind(len(edges))

    for v1, v2 in edges:
        if not graph.union(v1, v2):
            return [v1, v2]

In [ ]:
def regions_by_slashes(grid):
    N = len(grid)
    find_union = UnionFind(4 * N * N)

    # Traversing the list
    for r, row in enumerate(grid):
        for c, val in enumerate(row):
            root = 4 * (r * N + c)

            if val in '/ ':

                # Connecting the north and west components of the box
                find_union.union(root + 0, root + 1)

                # Connecting the east and south componeents of the box
                find_union.union(root + 2, root + 3)

            if val in '\ ':

                # Connecting the north and east components of the box
                find_union.union(root + 0, root + 2)

                # Connecting the west and south components of the box
                find_union.union(root + 1, root + 3)

            # Connecting the south component of the current box with the north component of the box below it
            if r+1 < N:
                find_union.union(root + 3, (root + 4 * N) + 0)

            # Connecting the north component of the current box with the south component of the box above it
            if r-1 >= 0:
                find_union.union(root + 0, (root - 4 * N) + 3)

            # Connecting the east component of the current box with the west component of the box on its right
            if c+1 < N:
                find_union.union(root + 2, (root + 4) + 1)

            # Connecting the west component of the current box with the east component of the box on its left
            if c-1 >= 0:
                find_union.union(root + 1, (root - 4) + 2)

    # Finding the number of connected components
    return sum(find_union.find(x) == x for x in range(4 * N * N))